In [1]:
import pandas as pd
import numpy as np
import tweepy
import time
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import SnowballStemmer
from gensim.models import CoherenceModel
tweets_all = pd.read_csv('Raw tweets_Jan6_2:40AM_CST_all.csv')
replies = pd.read_csv('Raw tweets_Jan6_2:40AM_CST_replies.csv')
text = tweets_all.append(replies).drop_duplicates(keep=False)
print('The number of tweets (excludes replies):',len(text))

The number of tweets (excludes replies): 6501


In [12]:
#text cleaning
import gensim
import gensim.corpora as corpora
import re
stemmer = SnowballStemmer('english')
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')
stop_words += ['cultured','clean','cultivated','meat','']
b = []
for i,u in text.iterrows():
    a = []
    word =''
    for words in str(u['tweets']).split():
        if '@' not in words: #remove @users
            if '#' not in words:
                if 'http' not in words: #remove URLs
                    if'&amp' not in words: #remove symbol
                        words = words.lower()# lower form
                        words = re.sub(r'[^a-zA-Z]', ' ', words) #replace non-alphabets characters with space. From "can't" to "can t"
                        if len(words)>3:
                            word += (words+' ')
    doc = ''
    for token in word.split():
        if token not in stop_words:
            if len(token)>2:
                token = porter.stem(token) #root form
                doc += (token+' ')
    b.append(doc)
text['processed']=[i for i in b]

In [13]:
#unigram
#unigram
unigram = []
unigram_list = []
for index, i in text.iterrows():
    unigram=[]
    for word in i['processed'].split():
        unigram.append(word)
    unigram_list.append(unigram)
data_words = [i for i in unigram_list]
id2word = corpora.Dictionary(data_words)
id2word.filter_extremes(no_below=3,no_above=0.3)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]

In [15]:
#unigram_number of topic = 10
from pprint import pprint
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=10, random_state=10,
                                            alpha='auto', per_word_topics=True)
pprint(lda_model.print_topics(num_words=20))

[(0,
  '0.035*"base" + 0.032*"food" + 0.023*"plant" + 0.019*"cell" + '
  '0.015*"industri" + 0.013*"innov" + 0.010*"year" + 0.010*"beyond" + '
  '0.010*"march" + 0.010*"seafood" + 0.009*"steak" + 0.009*"startup" + '
  '0.008*"invest" + 0.008*"good" + 0.008*"first" + 0.008*"make" + '
  '0.006*"panel" + 0.006*"product" + 0.006*"compani" + 0.006*"read"'),
 (1,
  '0.013*"steak" + 0.010*"want" + 0.009*"compani" + 0.009*"cell" + '
  '0.009*"space" + 0.009*"make" + 0.009*"world" + 0.009*"great" + 0.008*"year" '
  '+ 0.007*"meat" + 0.007*"week" + 0.007*"futur" + 0.007*"next" + '
  '0.006*"price" + 0.006*"anim" + 0.006*"food" + 0.006*"seem" + '
  '0.006*"contamin" + 0.006*"thank" + 0.006*"investor"'),
 (2,
  '0.018*"thank" + 0.013*"food" + 0.010*"livestock" + 0.009*"billion" + '
  '0.009*"free" + 0.009*"anim" + 0.009*"industri" + 0.009*"product" + '
  '0.009*"fish" + 0.008*"world" + 0.008*"global" + 0.008*"like" + 0.008*"work" '
  '+ 0.008*"sustain" + 0.008*"sale" + 0.007*"futur" + 0.007*"produ

In [20]:
#unigram_number of topic = 20
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=20, random_state=10,
                                            alpha='auto', per_word_topics=True)
pprint(lda_model.print_topics(num_words=20,num_topics=20))

[(0,
  '0.055*"base" + 0.047*"food" + 0.035*"cell" + 0.021*"plant" + '
  '0.019*"industri" + 0.017*"year" + 0.014*"protein" + 0.014*"startup" + '
  '0.013*"invest" + 0.011*"panel" + 0.011*"innov" + 0.011*"share" + '
  '0.010*"anim" + 0.009*"thought" + 0.009*"live" + 0.009*"sausag" + '
  '0.009*"rais" + 0.009*"entrepreneur" + 0.008*"think" + 0.008*"compani"'),
 (1,
  '0.030*"beef" + 0.016*"price" + 0.016*"kind" + 0.015*"join" + 0.014*"prof" + '
  '0.012*"mission" + 0.012*"post" + 0.011*"want" + 0.010*"amaz" + '
  '0.010*"regist" + 0.010*"pork" + 0.009*"edit" + 0.009*"cell" + 0.009*"busi" '
  '+ 0.009*"made" + 0.009*"scientif" + 0.009*"limit" + 0.008*"thank" + '
  '0.008*"compani" + 0.008*"founder"'),
 (2,
  '0.022*"podcast" + 0.020*"billion" + 0.016*"thank" + 0.015*"futur" + '
  '0.015*"number" + 0.015*"food" + 0.012*"speak" + 0.012*"listen" + '
  '0.012*"sector" + 0.012*"avail" + 0.011*"learn" + 0.011*"larg" + 0.010*"ceo" '
  '+ 0.010*"industri" + 0.009*"rise" + 0.009*"confer" + 0.009*

In [22]:
#unigram_number of topic = 30
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=30, random_state=10,
                                            alpha='auto', per_word_topics=True)
pprint(lda_model.print_topics(num_words=20,num_topics=30))

[(0,
  '0.087*"base" + 0.079*"plant" + 0.031*"cell" + 0.028*"food" + 0.028*"year" + '
  '0.023*"thing" + 0.021*"industri" + 0.019*"startup" + 0.019*"think" + '
  '0.017*"first" + 0.017*"market" + 0.014*"rais" + 0.013*"last" + '
  '0.011*"invest" + 0.011*"sector" + 0.009*"transpar" + 0.009*"seafood" + '
  '0.009*"featur" + 0.008*"start" + 0.008*"know"'),
 (1,
  '0.033*"join" + 0.023*"price" + 0.020*"ceo" + 0.020*"futur" + 0.017*"cell" + '
  '0.016*"speaker" + 0.016*"panel" + 0.014*"stem" + 0.014*"convers" + '
  '0.013*"advanc" + 0.013*"includ" + 0.012*"discuss" + 0.012*"industri" + '
  '0.011*"novemb" + 0.011*"programm" + 0.011*"today" + 0.011*"event" + '
  '0.010*"greenhous" + 0.010*"host" + 0.009*"regist"'),
 (2,
  '0.020*"make" + 0.019*"livestock" + 0.018*"product" + 0.017*"offer" + '
  '0.017*"number" + 0.016*"sustain" + 0.015*"free" + 0.015*"agricultur" + '
  '0.015*"demand" + 0.014*"accord" + 0.014*"larg" + 0.013*"gfi" + '
  '0.013*"protein" + 0.012*"industri" + 0.012*"consid" + 0

In [23]:
#unigram_number of topic = 40
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=40, random_state=10,
                                            alpha='auto', per_word_topics=True)
pprint(lda_model.print_topics(num_words=20,num_topics=40))

[(0,
  '0.038*"beyond" + 0.038*"protein" + 0.038*"base" + 0.029*"differ" + '
  '0.029*"food" + 0.023*"thought" + 0.020*"studi" + 0.019*"cell" + '
  '0.018*"altern" + 0.018*"landscap" + 0.018*"space" + 0.016*"call" + '
  '0.014*"capit" + 0.013*"make" + 0.013*"industri" + 0.012*"five" + '
  '0.011*"substitut" + 0.011*"improv" + 0.010*"investor" + 0.009*"august"'),
 (1,
  '0.042*"pandem" + 0.031*"engin" + 0.022*"caught" + 0.021*"avail" + '
  '0.020*"done" + 0.019*"team" + 0.018*"learn" + 0.017*"grown" + '
  '0.016*"increasingli" + 0.014*"meat" + 0.013*"passion" + 0.013*"board" + '
  '0.012*"stori" + 0.012*"lab" + 0.011*"consum" + 0.011*"relat" + '
  '0.010*"steak" + 0.010*"hire" + 0.010*"take" + 0.010*"scientist"'),
 (2,
  '0.025*"speak" + 0.021*"futur" + 0.021*"ceo" + 0.020*"everi" + '
  '0.019*"standard" + 0.018*"number" + 0.018*"june" + 0.018*"think" + '
  '0.016*"applic" + 0.016*"look" + 0.016*"categori" + 0.015*"industri" + '
  '0.015*"countri" + 0.015*"covid" + 0.015*"last" + 0.015*

In [24]:
#unigram_number of topic = 50
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=50, random_state=10,
                                            alpha='auto', per_word_topics=True)
pprint(lda_model.print_topics(num_words=20,num_topics=50))

[(0,
  '0.073*"march" + 0.062*"differ" + 0.052*"thing" + 0.030*"nation" + '
  '0.029*"landscap" + 0.027*"food" + 0.025*"approach" + 0.021*"good" + '
  '0.021*"call" + 0.020*"make" + 0.019*"space" + 0.017*"latest" + '
  '0.014*"respons" + 0.014*"obsolet" + 0.014*"back" + 0.013*"innov" + '
  '0.013*"label" + 0.011*"watch" + 0.011*"morn" + 0.011*"intern"'),
 (1,
  '0.073*"grown" + 0.062*"lab" + 0.051*"space" + 0.034*"think" + 0.033*"egg" + '
  '0.033*"first" + 0.030*"like" + 0.024*"beef" + 0.022*"cell" + 0.019*"anim" + '
  '0.019*"product" + 0.017*"compani" + 0.016*"kind" + 0.016*"base" + '
  '0.015*"partner" + 0.015*"develop" + 0.014*"meat" + 0.014*"steak" + '
  '0.014*"sold" + 0.012*"meet"'),
 (2,
  '0.042*"beef" + 0.040*"risk" + 0.024*"fast" + 0.021*"ceo" + 0.018*"associ" + '
  '0.018*"call" + 0.017*"learn" + 0.017*"covid" + 0.017*"futur" + 0.017*"food" '
  '+ 0.016*"public" + 0.015*"expand" + 0.014*"health" + 0.014*"novemb" + '
  '0.014*"grate" + 0.014*"etc" + 0.014*"complet" + 0.014*

In [25]:
#unigram_number of topic = 60
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=60, random_state=10,
                                            alpha='auto', per_word_topics=True)
pprint(lda_model.print_topics(num_words=20,num_topics=60))

[(0,
  '0.060*"rais" + 0.056*"fund" + 0.040*"seri" + 0.039*"talk" + 0.038*"start" + '
  '0.029*"founder" + 0.024*"food" + 0.022*"invest" + 0.022*"post" + '
  '0.022*"check" + 0.020*"entrepreneur" + 0.019*"interview" + 0.018*"join" + '
  '0.017*"innov" + 0.017*"startup" + 0.017*"tomorrow" + 0.015*"announc" + '
  '0.014*"leader" + 0.014*"morn" + 0.014*"sector"'),
 (1,
  '0.051*"kind" + 0.050*"lab" + 0.044*"requir" + 0.043*"less" + 0.038*"grown" '
  '+ 0.035*"first" + 0.030*"visual" + 0.029*"thank" + 0.026*"farm" + '
  '0.025*"near" + 0.024*"give" + 0.023*"would" + 0.018*"debat" + 0.018*"anim" '
  '+ 0.018*"develop" + 0.015*"coverag" + 0.015*"run" + 0.014*"turkey" + '
  '0.013*"much" + 0.013*"methan"'),
 (2,
  '0.053*"futur" + 0.053*"speak" + 0.044*"food" + 0.038*"muscl" + 0.030*"join" '
  '+ 0.028*"francisco" + 0.027*"number" + 0.026*"excit" + 0.022*"allow" + '
  '0.022*"june" + 0.019*"covid" + 0.018*"founder" + 0.017*"confer" + '
  '0.016*"team" + 0.016*"today" + 0.015*"protein" + 0.015